In [45]:
import pandas as pd
import ast
import numpy as np
from sklearn.metrics import roc_auc_score

df = pd.read_csv('test_2.csv')
df

,Unnamed: 0,smiles,label,sn2_states,e2_states
0,0,[N:1]#[C:2][C@@H:3]([NH2:4])[CH2:5][Br:6].[H-:7],sn2,"[[Energy(0.01092 Ha), 0, 'No imaginary frequen...","[[0, 0, 'no rc made']]"
1,1,[CH3:1][C@@H:2]([NH2:3])[CH2:4][Cl:5].[F-:6],sn2,"[[Energy(0.0179 Ha), Frequency(-133.40845 cm^-...","[[Energy(0.00246 Ha), Frequency(-46.53485 cm^-..."
2,2,[CH3:1][C@@H:2]([NH2:3])[CH2:4][Br:5].[Br-:6],e2,"[[Energy(0.00694 Ha), Frequency(-53.80338 cm^-...","[[0, 0, 'no rc made']]"
3,3,[CH3:1][CH2:2][C@H:3]([Br:4])[C:5]#[N:6].[Cl-:7],e2,"[[Energy(0.02081 Ha), 0, 'Hessian calc failed'...","[[Energy(0.005 Ha), Frequency(-38.42143 cm^-1)..."
4,4,[CH3:1][C@@:2]([Cl:3])([C:4]#[N:5])[N+:6](=[O:...,sn2,"[[Energy(0.01899 Ha), Frequency(-110.48387 cm^...","[[Energy(0.00444 Ha), Frequency(-63.1352 cm^-1..."
...,...,...,...,...,...
785,785,[CH3:1][C@@H:2]([C:3]#[N:4])[C@:5]([CH3:6])([N...,sn2,"[[Energy(0.06131 Ha), Frequency(-189.69135 cm^...","[[Energy(0.00032 Ha), 0, 'No imaginary frequen..."
786,786,[N:1]#[C:2][CH2:3][C:4]([NH2:5])([NH2:6])[F:7]...,sn2,"[[Energy(0.00296 Ha), Frequency(-291.50045 cm^...","[[Energy(0.00095 Ha), Frequency(-90.79349 cm^-..."
787,787,[N:1]#[C:2][CH2:3][C:4]([NH2:5])([NH2:6])[Cl:7...,sn2,"[[Energy(0.00073 Ha), Frequency(-56.55091 cm^-...","[[Energy(0.00145 Ha), Frequency(-177.3576 cm^-..."
788,788,[CH3:1][C@@H:2]([C:3]#[N:4])[C:5]([NH2:6])([NH...,sn2,"[[Energy(0.01251 Ha), 0, 'No imaginary frequen...","[[Energy(0.00195 Ha), Frequency(-197.02701 cm^..."


In [59]:
def parse_list(s):
    items = s.split(',')
    items = [x.strip() for x in items]

    tot_items = []
    for item in items:
        if item[0] == '[':
            tot_items.append([])
        tot_items[-1].append(item.strip('[]'))
    
    return tot_items

def parse_energy(energy_string):
    return float(energy_string.split('(')[-1].split(' ')[0])

def parse_frequency(frequency_string):
    return float(frequency_string.split('(')[-1].split(' ')[0])

In [62]:
parse_list(df['sn2_states'].values[0])

[['Energy(0.01092 Ha)', '0', "'No imaginary frequencies'"],
 ['Energy(0.01355 Ha)', '0', "'No imaginary frequencies'"],
 ['Energy(0.00129 Ha)', 'Frequency(-41.16208 cm^-1)', "''"],
 ['Energy(0.01092 Ha)', '0', "'No imaginary frequencies'"],
 ['Energy(0.01092 Ha)', 'Frequency(-55.5084 cm^-1)', "''"],
 ['Energy(0.00129 Ha)', 'Frequency(-26.0504 cm^-1)', "''"],
 ['Energy(0.01092 Ha)', '0', "'No imaginary frequencies'"]]

### Use all datapoints

In [51]:
def get_reaction_barriers(items, threshold = None):
    barriers = []
    for item in items:
        if item[0] != '0':
            barriers.append(parse_energy(item[0]))
    return barriers

suc = 0
preds = []
targets = []
for idx in range(len(df)):
    sn2_barriers = get_reaction_barriers(parse_list(df['sn2_states'].values[idx]))
    e2_barriers = get_reaction_barriers(parse_list(df['e2_states'].values[idx]))
    
    if len(sn2_barriers) > 0 and len(e2_barriers) > 0:
        suc += 1
        targets.append(df['label'].values[idx] == 'sn2')
        preds.append(np.mean(np.array(sn2_barriers)) < np.mean(np.array(e2_barriers)))
        
print(suc, len(df))
print(roc_auc_score(targets, preds))

585 790
0.5164023827502308


### use only reactions with succesfull hessian calc with im frequencies

In [54]:
def get_reaction_barriers(items, threshold = None):
    barriers = []
    for item in items:
        if item[-1] == "''":
            barriers.append(parse_energy(item[0]))
    return barriers

suc = 0
preds = []
targets = []
for idx in range(len(df)):
    sn2_barriers = get_reaction_barriers(parse_list(df['sn2_states'].values[idx]))
    e2_barriers = get_reaction_barriers(parse_list(df['e2_states'].values[idx]))
    
    if len(sn2_barriers) > 0 and len(e2_barriers) > 0:
        suc += 1
        targets.append(df['label'].values[idx] == 'sn2')
        preds.append(np.mean(np.array(sn2_barriers)) < np.mean(np.array(e2_barriers)))
        
print(suc, len(df))
print(roc_auc_score(targets, preds))

527 790
0.5235874244902046


### use only reaction with TS's with very large imaginary frequencies

In [70]:
def get_reaction_barriers(items, threshold = None):
    barriers = []
    for item in items:
        if item[-1] == "''" and parse_frequency(item[1]) < -80:
            barriers.append(parse_energy(item[0]))
    return barriers

suc = 0
preds = []
targets = []
for idx in range(len(df)):
    sn2_barriers = get_reaction_barriers(parse_list(df['sn2_states'].values[idx]))
    e2_barriers = get_reaction_barriers(parse_list(df['e2_states'].values[idx]))
    
    if len(sn2_barriers) > 0 and len(e2_barriers) > 0:
        suc += 1
        targets.append(df['label'].values[idx] == 'sn2')
        preds.append(np.min(np.array(sn2_barriers)) < np.max(np.array(e2_barriers)))
        
print(suc, len(df))
print(roc_auc_score(targets, preds))

211 790
0.4535147392290249
